In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

In [4]:
df = pd.read_csv('dataset/pima_indians_diabetes.csv')
df.head()

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [5]:
len(df)

767

In [6]:
df.isnull().sum()

6        0
148      0
72       0
35       0
0        0
33.6     0
0.627    0
50       0
1        0
dtype: int64

In [7]:
df.columns = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']
df.head()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,label
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [8]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
X.shape, y.shape

((767, 8), (767,))

In [10]:
# train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((613, 8), (154, 8), (613,), (154,))

In [11]:
# scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Data should be converted to PyTorch tensors first. One reason is that PyTorch usually operates in a 32-bit floating point while NumPy, by default, uses a 64-bit floating point. Mix-and-match is not allowed in most operations. Converting to PyTorch tensors can avoid the implicit conversion that may cause problems.

In [12]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([613, 8]),
 torch.Size([154, 8]),
 torch.Size([613]),
 torch.Size([154]))

In [14]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [26]:
class PrimaClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PrimaClassifier, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.layers(x)

In [44]:
# model parameters
input_size = 8
hidden_size = 16
output_size = 1
learning_rate = 0.01
epochs = 100
batch_size = 16

In [45]:
# initialize model
model = PrimaClassifier(input_size, hidden_size, output_size).to(device)
model

PrimaClassifier(
  (layers): Sequential(
    (0): Linear(in_features=8, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [46]:
# loss function and optimizer
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [47]:
# train
for epoch in range(epochs):
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i+batch_size].to(device)
        y_batch = y_train[i:i+batch_size].to(device)
        
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch.view(-1, 1))
        
        # backpropagation
        # optimizer.zero_grad() : it is important to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
        # loss.backward() : computes dloss/dx for every parameter x which has requires_grad=True. These are accumulated into x.grad for every parameter x.
        # optimizer.step() : causes the optimizer to take a step based on the gradients of the parameters.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if epoch % 10 == 0:
        print(f'Epoch: {epoch+1}, Loss: {loss.item():.4f}')

Epoch: 1, Loss: 0.3377
Epoch: 11, Loss: 0.2569
Epoch: 21, Loss: 0.2559
Epoch: 31, Loss: 0.2376
Epoch: 41, Loss: 0.2065
Epoch: 51, Loss: 0.1894
Epoch: 61, Loss: 0.1800
Epoch: 71, Loss: 0.1792
Epoch: 81, Loss: 0.1429
Epoch: 91, Loss: 0.1424


In [48]:
# train accuracy
with torch.no_grad():
    y_pred = model(X_train)
    y_pred = y_pred.round()
    acc = y_pred.eq(y_train.view_as(y_pred)).sum() / float(len(y_train))
    print(f'Train accuracy: {acc:.4f}')

Train accuracy: 0.9233


In [49]:
# test accuracy
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = y_pred.round()
    acc = y_pred.eq(y_test.view_as(y_pred)).sum() / float(len(y_test))
    print(f'Test accuracy: {acc:.4f}')

Test accuracy: 0.7143


In [50]:
# make predictions on the test set
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = y_pred.round()

    for i in range(len(y_test)):
        print(f'Data: {i+1} -> Predicted: {int(y_pred[i])}, Truth: {int(y_test[i])}')

Data: 1 -> Predicted: 0, Truth: 1
Data: 2 -> Predicted: 0, Truth: 1
Data: 3 -> Predicted: 1, Truth: 0
Data: 4 -> Predicted: 1, Truth: 1
Data: 5 -> Predicted: 0, Truth: 0
Data: 6 -> Predicted: 1, Truth: 0
Data: 7 -> Predicted: 1, Truth: 0
Data: 8 -> Predicted: 0, Truth: 1
Data: 9 -> Predicted: 0, Truth: 0
Data: 10 -> Predicted: 0, Truth: 0
Data: 11 -> Predicted: 0, Truth: 0
Data: 12 -> Predicted: 0, Truth: 1
Data: 13 -> Predicted: 0, Truth: 1
Data: 14 -> Predicted: 0, Truth: 0
Data: 15 -> Predicted: 0, Truth: 0
Data: 16 -> Predicted: 0, Truth: 0
Data: 17 -> Predicted: 1, Truth: 0
Data: 18 -> Predicted: 1, Truth: 0
Data: 19 -> Predicted: 0, Truth: 0
Data: 20 -> Predicted: 0, Truth: 0
Data: 21 -> Predicted: 1, Truth: 1
Data: 22 -> Predicted: 1, Truth: 0
Data: 23 -> Predicted: 1, Truth: 0
Data: 24 -> Predicted: 0, Truth: 0
Data: 25 -> Predicted: 0, Truth: 0
Data: 26 -> Predicted: 0, Truth: 1
Data: 27 -> Predicted: 1, Truth: 0
Data: 28 -> Predicted: 0, Truth: 0
Data: 29 -> Predicted: 0, Tru